In [1]:
import pandas as pd

import requests

import bs4
from bs4 import BeautifulSoup

In [2]:
url_template = "https://www.nationtv.tv/main/content/{}"
latest_news_id = 378803102

In [3]:
save_path = "nation_politics_{}.csv"

In [4]:
def get_news_from_url(news_id):
    response = requests.get(url_template.format(news_id))
    html_body_str = response.text
    return html_body_str

In [5]:
body_str = get_news_from_url(latest_news_id - 999)

In [6]:
soup = BeautifulSoup(body_str, 'html.parser')

In [7]:
soup

<!DOCTYPE html>

<html dir="ltr" lang="en">
<head>
<title>หลายหน่วยงานรุดช่วยหนุ่มกรีดยางถูกฟ้อง  97 ล้าน</title>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, height=device-height, initial-scale=1" name="viewport"/>
<meta content="ฟ้องล้มละลาย,บุรีรัมย์,หนุ่มสวนยาง " name="keywords"/>
<meta content="ผอ.บังคับคดีบุรีรัมย์ พร้อมผู้แทนหลายหน่วยงาน ลงพื้นที่ช่วยเหลือหนุ่มรับจ้างกรีดยางมีชื่อโผล่เป็นผู้ถือหุ้นบริษัทอสังหาริมทรัพย์ จนถูกฟ้องล้มละลายเป็นหนี้กว่า 97 ล้าน เบื้องต้นได้บันทึกคำให้การส่งไปศาลล้มละลายกลาง เพื่อยืนยันว่าไม่ได้มีส่วนเกี่ยวข้องกับบริษัทที่ถูกฟ้อง คาดอาจทำเอกสารหายหรือมีมิจฉาชีพนำไปใช้ พร้อมฝากผู้นำหมู่บ้านเตือน ปชช.ระมัดระวังการใช้เอกสารส่วนตัว" name="description"/>
<link href="https://resource.nationtv.tv/assets/images/icon/apple-icon-57x57.png" rel="apple-touch-icon" sizes="57x57"/>
<link href="https://resource.nationtv.tv/assets/images/icon/apple-icon-60x60.png" rel="apple-touch-icon" siz

In [8]:
def get_article_category(soup):
    divs = soup.findAll("div", {"class": "article-category --agents-category"})
    categ = divs[0]['data-category']
    return categ

def get_article_date(soup):
    divs = soup.findAll("div", {"class": "article-date"})
    date_str = divs[0].string
    return date_str

def get_article_preface(soup):
    divs = soup.findAll("h2", {"class": "article-preface"})
    preface_str = divs[0].string
    return preface_str

def get_article_body(soup):
    body = soup.findAll("div", {"class": "article-body"})[0]
    body_parts = []

    for elm in body.children:
        if type(elm) == bs4.element.Tag:
            content = elm.string
            if content:
                if content.strip() != "":
                    body_parts.append(content.strip())
    return "\n".join(body_parts)

In [9]:
get_article_category(soup), get_article_date(soup), get_article_preface(soup)

('ภูมิภาค',
 '21 ต.ค. 2563',
 'ผอ.บังคับคดีบุรีรัมย์ พร้อมผู้แทนหลายหน่วยงาน ลงพื้นที่ช่วยเหลือหนุ่มรับจ้างกรีดยางมีชื่อโผล่เป็นผู้ถือหุ้นบริษัทอสังหาริมทรัพย์ จนถูกฟ้องล้มละลายเป็นหนี้กว่า 97 ล้าน เบื้องต้นได้บันทึกคำให้การส่งไปศาลล้มละลายกลาง เพื่อยืนยันว่าไม่ได้มีส่วนเกี่ยวข้องกับบริษัทที่ถูกฟ้อง คาดอาจทำเอกสารหายหรือมีมิจฉาชีพนำไปใช้ พร้อมฝากผู้นำหมู่บ้านเตือน ปชช.ระมัดระวังการใช้เอกสารส่วนตัว')

In [10]:
def parse_article(soup):
    title = soup.title.string
    date = get_article_date(soup)
    preface = get_article_preface(soup)
    body = get_article_body(soup)
    article = {
        'title': title,
        'date': date,
        'preface': preface,
        'body': body
    }
    return article

In [26]:
def crawl_nation_articles(start_news_id, n_limits=2000, allowed_categs=[]):
    news_id = start_news_id
    while len(articles) < n_limits:
        if len(articles) % 100 == 0 and articles:
            print("{} articles fetched".format(len(articles)))
        html_body_str = get_news_from_url(news_id)
        soup = BeautifulSoup(html_body_str, 'html.parser')
        try:
            categ = get_article_category(soup)
        except:
            print("Not found. Cannot find title")
            news_id -= 1
            continue
        
        if categ in allowed_categs:
            try:
                article = parse_article(soup)
            except:
                print("Article parse error")
                news_id -= 1
                continue
            article['id'] = news_id
            article['category'] = categ
            articles.append(article)
        
        news_id -= 1

In [12]:
allowed_categs = ['การเมือง', 'ข่าวในพระราชสำนัก']
n_limits = 2000

In [ ]:
articles = []
articles = crawl_nation_articles(latest_news_id, n_limits=n_limits, allowed_categs=allowed_categs)

In [14]:
articles[0]

{'title': 'ทนายเตรียมยื่นอุทธรณ์คำสั่งไม่ให้ประกัน อานนท์',
 'date': '27 ต.ค. 2563',
 'preface': 'ทนายเตรียมยื่นอุทธรณ์คำสั่งไม่ให้ประกัน "อานนท์" ยกบรรทัดฐานศาลอุทธรณ์ภาค 5 ให้ประกันยื่นใหม่พรุ่งนี้ ลุ้นไต่สวน "เพนกวิน" ละเมิดอำนาจศาล เบิกตัว "อานนท์" ขึ้นว่าความ',
 'body': 'ผู้สื่อข่าวรายงานว่า ในวันพรุ่งนี้ (28 ต.ค.) ที่จะมีการไต่สวน นายพริษฐ์ หรือเพนกวิน ละเมิดอำนาจศาล จะมีการออกหมายเรียกนายอานนท์จากเรือนจำในฐานะทนายความนายพริษฐ์ มาว่าความด้วยเนื่องจากตัวนายอานนท์เองยังไม่ได้ถูกเพิกถอนใบอนุญาตว่าความ ยังสามารถว่าความตามกฎหมายได้อยู่\nสำหรับรายละเอียดไต่สวนคดีละเมิดอำนาจศาลนั้น เวลา 09.00 น.ที่ห้องพิจารณาคดี 710 ศาลอาญา ถ.รัชดาภิเษก ศาลนัดไต่สวนคดีละเมิดอำนาจศาลหมายเลขดำ ลศ. 9/2563 ที่ผอ.สำนักอำนวยการประจำศาลอาญา กล่าวหานายพริษฐ์ หรือเพนกวิน ชิวารักษ์ แกนนำมวลชนกลุ่ม ประชาชนปลดแอก ผู้ถูกกล่าวหาเรื่องละเมิดอำนาจศาล\nกรณีเมื่อเวลา 09.00 น.วันที่ 8 ส.ค.2563 ขณะที่พนักงานสอบสวน สน.สำราญราษฎร์ นำตัวนายอานนท์ นำภา และนายภาณุพงศ์ จาดนอก หรือไมค์ ระยอง 2 ผู้ต้องหา คดี ปราศรัยปลุกปั่นยุยง ฯ 

In [30]:
df_nation_news = pd.DataFrame(articles)

In [21]:
def normalize_date(date_th_str):
    day, month, year = date_th_str.split()
    months_th = ["มค", "กพ" , "มีค" , "เมย" , "พค", "มิย", 
              "กค", "สค", "กย", "ตค", "พย", "ธค"]
    month_num = months_th.index(month.replace(".", "")) + 1
    year_ad = int(year) - 543
    date_fmt = "{}/{}/{}".format(year_ad, str(month_num).zfill(2), day.zfill(2))
    return date_fmt

In [20]:
df_nation_news_bk = df_nation_news.copy()

In [31]:
df_nation_news['date'] = df_nation_news['date'].apply(normalize_date)

In [32]:
df_nation_news.to_csv(save_path.format(len(articles)), index=False)

In [33]:
df_nation_news.tail()

,title,date,preface,body,id,category
3995,ปชป.เปิดตัว2กระบอกเสียงพรรคใหม่เน้นการเมือง-ปั...,2020/05/16,"""ประชาธิปัตย์"" ถือฤกษ์เปิดตัว 2 ส.ส.รุ่นใหม่ ท...",(16 พฤษภาคม 2563) นายราเมศ รัตนะเชวง โฆษกพรรคป...,378776897,การเมือง
3996,จัดหนัก!พาณิชย์จับมือห้างใหญ่ลดราคาสินค้า6กลุ่...,2020/05/16,"""ประชาธิปัตย์""ชี้แนวโน้มเศรษฐกิจไทยเตรียมดีขึ้...","ทั้งนี้ แนวทางของนายจุรินทร์ ได้วางไว้ คือ ""เก...",378776886,การเมือง
3997,เปิดขั้นตอนลงทะเบียนร้านค้าผ่าน WWW.ไทยชนะ.com,2020/05/16,เปิดขั้นตอนลงทะเบียนร้านค้าผ่าน www.ไทยชนะ.com...,1.ลงทะเบียนเข้าร่วมมาตรการ เข้าสู่เว็บไซต์ อ่า...,378776883,การเมือง
3998,เทพไทหนุนสังคมไทยร่วมแบ่งทุกข์คนยากผ่านตู้ปันสุข,2020/05/16,"""เด็กประชาธิปัตย์""หนุนแนวคิด""ตู้ปันสุข""เพื่อบร...",อย่างไรก็ตาม จึงได้ปรับวิธีการช่วยเหลือประชาชน...,378776879,การเมือง
3999,ศบค. ย้ำยังไม่อนุญาตเข้าประเทศ หลังปลดจีน-เกาห...,2020/05/16,"""ศูนย์โควิด"" แถลงไม่พบผู้ติดเชื้อใหม่ แจง แพลต...",นพ.ทวีศิลป์ วิษณุโยธิน โฆษกศูนย์บริหารสถานการณ...,378776876,การเมือง


In [28]:
batch_2_start_id = 378791777 - 1
crawl_nation_articles(batch_2_start_id, n_limits=4000, allowed_categs=allowed_categs)

2000 articles fetched
2000 articles fetched
Not found. Cannot find title
Not found. Cannot find title
Not found. Cannot find title
Not found. Cannot find title
Not found. Cannot find title
Not found. Cannot find title
Not found. Cannot find title
Not found. Cannot find title
Not found. Cannot find title
Not found. Cannot find title
Not found. Cannot find title
Not found. Cannot find title
Not found. Cannot find title
Not found. Cannot find title
Not found. Cannot find title
Not found. Cannot find title
Not found. Cannot find title
Not found. Cannot find title
Not found. Cannot find title
Not found. Cannot find title
Not found. Cannot find title
Not found. Cannot find title
Not found. Cannot find title
Not found. Cannot find title
Not found. Cannot find title
Not found. Cannot find title
Not found. Cannot find title
Not found. Cannot find title
Not found. Cannot find title
Not found. Cannot find title
Not found. Cannot find title
Not found. Cannot find title
Not found. Cannot find title

In [29]:
len(articles)

4000